### Text of Frey and Dueck (2007)

Following the Supporting Online Material, page 5-6.

"For each sentence in the main text of this manuscript, words delimited by spaces were extracted, punctuation was removed, and words with fewer than 5 characters were discarded. The similarity of sentence i to sentence k was set to the negative sum of the information-theoretic costs (S5) of encoding every word in sentence i using the words in sentence k and a dictionary of all words in the manuscript. For each word in sentence i, if the word matched a word in sentence k, the coding cost for the word was set to the negative logarithm of the number of words in sentence k (the cost of coding the index of the matched word), and otherwise it was set to the negative logarithm of the number of words in the manuscript dictionary (the cost of coding the index of the word in the manuscript dictionary). A word was considered to match another word if either word was a substring of the other."

#### import Affinity Propagation Source Code

In [1]:
AffinityPackage_import_type=['load_installed_package','load_from_directory'][0]

In [2]:
if AffinityPackage_import_type=='load_installed_package':
    """pip install -i https://test.pypi.org/simple/ AffinityPropagation-RezaLevin2020-pkg-DukePhDs==0.0.3"""
    from AffinityPropagation_RezaLevin2020_pkg import Src_AP_V13 as AfP
else:
    """can be found in this repository: https://github.com/MReza89/Stat663_Spring2020_FinalProject_RezaLevin"""
    import Src_AP_V13 as AfP # requires to have Src_AP_V13.py in the current directory

#### import libraries related to testing

In [3]:
#import sklearn.cluster as cluster
import numpy as np
#from sklearn_extra.cluster import KMedoids

import time
import string

##### Data 

In [4]:
with open('paper_FreyDueck2007.txt') as file:
    text = file.read()

In [5]:
import re
sentences = [(i, sentence) for i,sentence in enumerate(re.split('\. |\n', text))]


In [6]:
[sentences[i] for i in np.random.choice(len(sentences),4)]

[(146,
  'Yet, to our knowledge, affinity propagation is the first method to make use of this idea to solve the age-old, fundamental problem of clustering data'),
 (89,
  'The reconstruction errors for affinity propagation and k-centers clustering are compared in Fig'),
 (55,
  'For point i, the value of k that maximizes a(i,k) + r(i,k) either identifies point i as an exemplar if k = i, or identifies the data point that is the exemplar for point i'),
 (27, '')]

In [7]:
sentences_key = dict(sentences)

In [8]:
sentences_stripped = [(i," ".join(sentence.split()).translate(str.maketrans("","",string.punctuation)).lower()) 
                      for i,sentence in enumerate(re.split('\. |\n', text))]

In [9]:
[sentences_stripped[i] for i in np.random.choice(len(sentences_stripped),4)]

[(110,
  'when the number of most accessible cities was constrained to be seven by adjusting the input preference appropriately the cities shown in fig'),
 (38,
  'in the first iteration because the availabilities are zero rik is set to the input similarity between point i and point k as its exemplar minus the largest of the similarities between point i and other candidate exemplars'),
 (51,
  'this message reflects accumulated evidence that point k is an exemplar based on the positive responsibilities sent to candidate exemplar k from other points'),
 (148, 'â© 2020 github inc')]

In [10]:
sentences_final = [(index, " ".join(list_of_words))
                   for (index, list_of_words) in [(index, [word for word in sentence_words if len(word) >= 5]) 
                                         for index,sentence_words in [(index,sentence.split()) 
                                                                      for index,sentence in sentences_stripped]] 
                   if len(list_of_words) > 0]


In [11]:
[sentences_final[i] for i in np.random.choice(len(sentences_final),4)]

[(123,
  'degenerate cases energy function multiple minima corresponding multiple fixed points update rules these prevent convergence'),
 (32,
  'availability candidate exemplar point point reflects accumulated evidence appropriate would point choose point exemplar taking account support other points point should exemplar'),
 (139,
  'there input assumed metric nonnegative symmetric satisfying triangle inequality'),
 (130,
  'these techniques improved methods begin large number clusters prune still random sampling pruning decisions cannot recovered')]

In [12]:
len(sentences_final)

125

In [13]:
sentences_final_data = [sentences for i,sentences in sentences_final]

#### Full dictionary

In [14]:
dictionary = set([word for words in [sentence.split() for i,sentence in sentences_final] for word in words])

In [15]:
len(dictionary)

626

In [16]:
i = sentences_final_data[0]
k = sentences_final_data[1]
i, k

('clustering identifying subset representative examples important processing sensory signals detecting patterns',
 'exemplars found randomly choosing initial subset points iteratively refining works initial choice close solution')

In [17]:
AfP.sentence_similarity(i, k, dictionary)

-67.03256104061624

In [18]:
# Check the answer. There's only one match ('subset') and sentence i has 11 words:
np.allclose(AfP.sentence_similarity(i, k, dictionary) ,
            -10*np.log(len(dictionary)) - np.log(len(k.split())))

True

Notice that sentence_similarity is not symmetric. Further, in our implementation, we automatically set sentence similarity to be 0 if the sentences are the same.

In [19]:
AfP.sentence_similarity(k, i, dictionary)

-86.10945009709967

In [20]:
AfP.sentence_similarity(i, i, dictionary)

0

#### Apply affinity propagation to the document

In [21]:
# Setting the preference
# Preference was set to the number of words in the sentence times the negative logarithm of the 
# number of words in the manuscript dictionary, plus some constant (set to 90 in the paper)
constant = -80
pref = [len(sentence.split())*(-np.log(len(dictionary))) + constant for sentence in sentences_final_data]

In [22]:
%%time
s_exemplars,s_labels,s_Cluster_Centers,s_last_iteration=AfP.affinity_propagation(sentences_final_data, 
                                                                             Similarity_metric_="Sentences",
                                                                             preference=pref, 
                                                                             max_iter=200, 
                                                                             Plot_Clusters=False,Dictionary=dictionary)
print("Number of exemplars: %i" % len(s_exemplars))

[15 22 38 59] 38
Number of exemplars: 4
Wall time: 597 ms


In [23]:
[sentences_final[i] for i in s_exemplars]

[(17,
  'affinity propagation takes input collection realvalued similarities between points where similarity indicates point index suited exemplar point'),
 (25,
  'priori points equally suitable exemplars preferences should common value value varied produce different numbers clusters'),
 (46,
  'availability selfresponsibility positive responsibilities candidate exemplar receives other points'),
 (70,
  'affinity propagation found exemplars lower squared error kcenters clustering')]

In [24]:
# Converting to full sentences
[sentences_key[key] for key in [i for i, sentence in [sentences_final[i] for i in s_exemplars]]]

['Affinity propagation takes as input a collection of real-valued similarities between data points, where the similarity s(i,k) indicates how well the data point with index k is suited to be the exemplar for data point i',
 'If a priori, all data points are equally suitable as exemplars, the preferences should be set to a common value- this value can be varied to produce different numbers of clusters',
 'The availability a(i,k) is set to the self-responsibility r(k,k) plus the sum of the positive responsibilities candidate exemplar k receives from other points',
 'Affinity propagation found exemplars with much lower squared error than the best of 100 runs of k-centers clustering (Fig']